# Base Code for Calculating Atom Movement

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets

In [84]:
# define the number of floating atoms
floaters = np.zeros(10*3)
n = len(floaters)
m = n/3

# set the positions of the floating atoms
count = 0
k = 1.7
for i in range(0,n,3):
    floaters[i] = count
    floaters[i+1] = k*count
    floaters[i+2] = 1
    count += 1

In [3]:
# set number of substrate atoms (substrate will be 2*ns + 1 side lengths)
ns = 5

In [4]:
# set the substrate parameters
h_x = 1
h_y = 1

In [5]:
# set the VDW parameters
w = 1
sigma = 1

In [6]:
# set the spring parameters
k_s = 5
l = 1

In [109]:
# define the time interval for the gradient flow
t = np.linspace(0,6,15)

In [116]:
# this is the RHS of the system of ODEs solved by odeint. It returns the forces
# acting on each floater in the x, y and z directions
def spring_and_VDWForce(floaters, t):

    # compute number of atoms
    m = int(len(floaters)/3)
    n = int(len(floaters))

    # split the atoms into 1x3 arrays
    r = [[0 for i in range(3)] for j in range(m)]
    for i in range(m):
        r[i] = floaters[i*3:i*3+3]

    # initialize arrays for the spring forces and VDW forces
    combined_spring_forces = np.zeros(n)
    combined_VDW_forces = np.zeros(n)
    VDW_force = np.zeros((m, 3))
    spring_force = np.zeros((m, 3))

    # define the size of the substrate surrounding the floaters
    a = np.arange(-ns,ns+1,1)

    # compute the distances of the floating atom from the substrate atoms
    # and the VDW forces acting on them as a result
    for j in range(len(a)):
        for i in range(len(a)):
            for k in range(m):

                # compute the offsets
                k_x = (r[k][0] + .5)%h_x
                k_y = (r[k][1] + .5)%h_y

                # compute dx, dy, dz and rhat
                dx = k_x - .5 + a[i]*h_x
                dy = k_y - .5 + a[j]*h_y
                dz = r[k][2]
                rhat = np.sqrt(dx**2 + dy**2 + dz**2)

                # this is the gradient of V (computed by hand), it is used to compute the
                # VDW forces acting on the floating atoms
                gradV_common = (12*w*((sigma**6)/(rhat**8)-(sigma**12)/(rhat**14)))
                VDW_force[k] = -gradV_common*np.array([dx, dy, dz])

            # combine the VDW forces for all floating atoms
            combined_VDW_forces += np.hstack(VDW_force)

    # this is the gradient of E_s (computed by hand), it is used to compute the
    # spring forces acting on the atoms
    for i in range(m-1):
        gradE_common = k_s*(norm(r[i]-r[i+1]) - l)/norm(r[i] - r[i+1])
        current_sf = -gradE_common*np.array([(r[i][0]-r[i+1][0]),(r[i][1]-r[i+1][1]),(r[i][2]-r[i+1][2])])
        spring_force[i] += current_sf
        spring_force[i+1] = -current_sf


    combined_spring_forces = np.hstack(spring_force)
    total_spring_and_VDW_force = np.add(combined_spring_forces, combined_VDW_forces)

    return total_spring_and_VDW_force


In [117]:
%%time
# compute the changing positions of the atoms as a result of the spring
# and VDW forces acting on them
gFlow = odeint(spring_and_VDWForce, floaters, t, atol=1.4e-10)

CPU times: user 1min 34s, sys: 1.07 s, total: 1min 36s
Wall time: 1min 38s


# Simulation Code

In [118]:
# define arrays of x, y and z coordinates to create an animated scatter plot
x = np.zeros((len(t),int(m)))
y = np.zeros((len(t),int(m)))
z = np.zeros((len(t),int(m)))
for i in range(len(t)):
    for j in range(int(m)):
        x[i][j] = gFlow[i,j*3]
        y[i][j] = gFlow[i,j*3+1]
        z[i][j] = gFlow[i,j*3+2]

In [123]:
# create an ipyvolume.pylab figure
p3.figure()

# plot a scatterplot of the arrays defined above in the figure
s1 = p3.scatter(x, y, z, marker='sphere')
s2 = p3.plot(x, y, z)

# set the x, y and z limits of the figure
p3.xlim(-4,12)
p3.ylim(-4,20)
p3.zlim(0,1)

# add animation controls to the figure
p3.animation_control(s1)
p3.animation_control(s2)

# add sliders for parameters
ns_slider = widgets.IntSlider(min=0, max=15, step=1, value=ns, description='ns', continuous_update=False)
w_slider = widgets.IntSlider(min=0, max=10, step=1, value=w, description='w', continuous_update=False)
sigma_slider = widgets.IntSlider(min=0, max=10, step=1, value=sigma, description='sigma', continuous_update=False)
k_s_text = widgets.FloatText(min=0., max=500., step=.1, value=k_s, description='k_s', continuous_update=False)
l_slider = widgets.IntSlider(min=0, max=10, step=1, value=l, description='l', continuous_update=False)

# create handler functions to change the values of the parameters globally when the sliders change
def on_ns_slider_change(change):
    global ns
    ns = change.new

def on_w_slider_change(change):
    global w
    w = change.new

def on_sigma_slider_change(change):
    global sigma
    sigma = sigma.new
    
def on_k_s_text_change(change):
    global k_s
    k_s = change.new
    
def on_l_slider_change(change):
    global l
    l = change.new

# set the sliders to call their handler functions any time the sliders are moved
ns_slider.observe(on_ns_slider_change, names='value')
w_slider.observe(on_w_slider_change, names='value')
sigma_slider.observe(on_sigma_slider_change, names='value')
k_s_text.observe(on_k_s_text_change, names='value')
l_slider.observe(on_l_slider_change, names='value')

label = widgets.Label('Parameter sliders:')
parameters = widgets.VBox([ipv.gcc(), label, ns_slider, w_slider, sigma_slider, l_slider, k_s_text])
display(parameters)

VBox(children=(VBox(children=(Figure(animation=200.0, animation_exponent=1.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', size=array(2), size_selected=array(2.6), x=array([[ 0.        ,  1.        ,  2.        ,  3.        ,  4.        ,
         5.        ,  6.        ,  7.        ,  8.        ,  9.        ],
       [ 0.26436567,  1.52281662,  1.78463491,  2.67124694,  4.23298073,
         5.07323329,  5.90431821,  7.44031665,  8.34223429,  8.49471   ],
       [ 0.51883908,  1.52404124,  2.24342216,  2.68620144,  4.36322552,
         5.32739785,  6.45505992,  7.53592253,  8.44933999,  8.49889653],
       [ 0.49628324,  1.53462333,  2.4821526 ,  2.77058548,  4.47956553,
         5.46369568,  6.50346483,  7.53817368,  8.46356216,  8.49941577],
       [ 0.49596174,  1.53443075,  2.4824913 ,  2.77098939,  4.47989796,
         5.46350175,  6.50324074,  7.53809909,  8.46390291,  8.49942701],
       [ 0.4959611 ,  1.53443044,  2.48249219,  2.77100768,  4.47990764,
         5.46349755,  6.50323587,  7.53809765,  8.46391109,  8.49942727],
       [ 0.49596112,  1.53443046,  2.48249216,  2.7710083 ,  4.47990795,
         5.46349742,  6.50323573,  7.53809761,  8.46391134,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728],
       [ 0.49596112,  1.53443046,  2.48249216,  2.77100832,  4.47990796,
         5.46349742,  6.50323572,  7.53809761,  8.46391135,  8.49942728]]), y=array([[  0.        ,   1.7       ,   3.4       ,   5.1       ,
          6.8       ,   8.5       ,  10.2       ,  11.9       ,
         13.6       ,  15.3       ],
       [  0.07911404,   1.62096299,   3.6058206 ,   5.40855387,
          6.67121189,   8.49776547,  10.32773373,  11.61545437,
         13.41176922,  15.41008225],
       [ -0.10012226,   1.6287988 ,   3.50215079,   5.45956349,
          6.56423034,   8.32454682,   9.97946654,  11.66221509,
         13.44898834,  15.45154404],
       [  0.02913336,   1.73737814,   3.50054552,   5.48052203,
          6.52304341,   8.26701901,   9.97502994,  11.68583336,
         13.46394791,  15.46603001],
       [  0.02995468,   1.73810195,   3.50091153,   5.4808482 ,
          6.5234506 ,   8.2677577 ,   9.9757729 ,  11.68646945,
         13.46430657,  15.46637643],
       [  0.02995529,   1.73810232,   3.50091116,   5.48084485,
          6.52345973,   8.26777534,   9.97579051,  11.68648454,
         13.46431515,  15.46638471],
       [  0.02995518,   1.73810222,   3.50091109,   5.48084467,
          6.52346   ,   8.26777588,   9.97579105,  11.68648501,
         13.46431542,  15.46638496],
       [  0.02995517,   1.73810221,   3.50091109,   5.48084466,
          6.52346001,   8.2677759 ,   9.97579107,  11.68648502,
         13.46431543,  15.46638497],
       [  0.02995517,   1.7381

In [120]:
# print the output of odeint
pprint.pprint(gFlow)

array([[  0.        ,   0.        ,   1.        ,   1.        ,
          1.7       ,   1.        ,   2.        ,   3.4       ,
          1.        ,   3.        ,   5.1       ,   1.        ,
          4.        ,   6.8       ,   1.        ,   5.        ,
          8.5       ,   1.        ,   6.        ,  10.2       ,
          1.        ,   7.        ,  11.9       ,   1.        ,
          8.        ,  13.6       ,   1.        ,   9.        ,
         15.3       ,   1.        ],
       [  0.26436567,   0.07911404,   0.92031471,   1.52281662,
          1.62096299,   0.74056701,   1.78463491,   3.6058206 ,
          0.83313   ,   2.67124694,   5.40855387,   0.79522957,
          4.23298073,   6.67121189,   0.85732449,   5.07323329,
          8.49776547,   0.84234772,   5.90431821,  10.32773373,
          0.88481661,   7.44031665,  11.61545437,   0.75737123,
          8.34223429,  13.41176922,   0.77893133,   8.49471   ,
         15.41008225,   0.72556415],
       [  0.51883908,  -0.1001

In [121]:
# compute and print the distance between each pair of bonded atoms
# for debugging purposes
for i in range(int(m)-1):
    dist = norm(gFlow[-1,i*3:i*3+3] - gFlow[-1,i*3+3:i*3+6])
    print(str(dist)+', ')

1.9996062308, 
2.00367827969, 
2.00334119623, 
2.00421018418, 
2.0044146293, 
2.00019927503, 
2.00051788125, 
2.00547437479, 
2.00239341342, 
